In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import random

In [2]:
df_sp = pd.read_csv(r'F:\Abhilah_First\Results\Era5Land\noised_stake_data.csv')
df_sp.head(4)

,Unnamed: 0,Julian Day,obs_period(days),elevation(m),Derbis thickness (cm),melt (cm),pdd,Model-1 Melt Rate (cm/day),Model-2 Melt Rate (cm/day),Model-3 Melt Rate (cm/day)
0,0,232,17,3868.0,116.704707,10.789589,178.524402,0.960872,0.522011,0.543095
1,1,250,18,3867.0,113.487625,5.006354,152.118696,0.773263,0.420089,0.437056
2,2,263,13,3867.0,114.203133,4.979968,100.791271,0.709409,0.385399,0.400965
3,3,273,10,3868.0,116.584423,10.297328,72.470200,0.663096,0.360239,0.374789


In [3]:
df_model1_param = pd.read_csv(r'F:\Abhilah_First\Results\Era5Land\Parameters_1000.csv')
df_model1_param.head(4)

,Unnamed: 0,TF1 Model 1,TF2 Model 1,MSE Model 1,DDF Model 2,d0 Model 2,MSE Model 2,DDF Model 3,d0 Model 3,a Model 3,MSE Model 3
0,0,1.098285,-0.328110,0.505590,4.286001,17.206201,0.437698,3.702612,11.623785,0.444232,0.427410
1,1,1.196568,-0.287907,0.504975,5.024441,12.488065,0.393736,4.370584,6.361340,0.350399,0.396229
2,2,1.111208,-0.328610,0.533966,4.615500,14.941232,0.454123,3.942050,9.816933,0.422546,0.450480
3,3,1.043782,-0.375285,0.481698,4.650498,15.363376,0.426325,3.825529,11.479997,0.446073,0.422991


In [4]:
data = {'TF1':df_model1_param['TF1 Model 1']} 
df_all_iterations = pd.DataFrame(data)
df_all_iterations['TF2'] = df_model1_param['TF2 Model 1']
MSEs = list(df_model1_param['MSE Model 1'])
df_all_iterations['MSE'] = df_model1_param['MSE Model 1']
weight = []
for i in MSEs:
    weight.append(math.exp(-i))
df_all_iterations['Weight'] = weight
df_all_iterations

,TF1,TF2,MSE,Weight
0,1.098285,-0.328110,0.505590,0.603149
1,1.196568,-0.287907,0.504975,0.603521
2,1.111208,-0.328610,0.533966,0.586275
3,1.043782,-0.375285,0.481698,0.617734
4,1.101841,-0.327316,0.478549,0.619682
...,...,...,...,...
995,1.092626,-0.336368,0.485270,0.615531
996,1.099246,-0.341052,0.509144,0.601010
997,1.062582,-0.328680,0.462533,0.629687
998,0.949135,-0.407123,0.391767,0.675861


In [5]:
print(min(weight), max(weight))
count = 0
for i in weight:
    if i <= 0.645:
        count +=1
print('count: ', count) 
# there is no zero weight, number of non zero weights is 1000

0.5051463215456174 0.7020802720471501
count:  896


In [6]:
df_all_iterations['TF1*Weight'] = df_all_iterations['TF1']*df_all_iterations['Weight']
df_all_iterations['TF2*Weight'] = df_all_iterations['TF2']*df_all_iterations['Weight']

df_all_iterations['Weight*(TF1 - xBARw)^2'] = df_all_iterations['Weight']*((df_all_iterations['TF1'] - sum(df_all_iterations['TF1*Weight']/1000) )**2)
df_all_iterations['Weight*(TF2 - yBARw)^2'] = df_all_iterations['Weight']*((df_all_iterations['TF2'] - sum(df_all_iterations['TF2*Weight']/1000) )**2)

df_all_iterations.head(4)

,TF1,TF2,MSE,Weight,TF1*Weight,TF2*Weight,Weight*(TF1 - xBARw)^2,Weight*(TF2 - yBARw)^2
0,1.098285,-0.328110,0.505590,0.603149,0.662430,-0.197899,0.100724,0.011054
1,1.196568,-0.287907,0.504975,0.603521,0.722154,-0.173758,0.155095,0.005467
2,1.111208,-0.328610,0.533966,0.586275,0.651473,-0.192656,0.104196,0.010824
3,1.043782,-0.375285,0.481698,0.617734,0.644780,-0.231826,0.077477,0.020586


In [7]:
TF1_variability = (sum(df_all_iterations['Weight*(TF1 - xBARw)^2'])*1000/(1000*(1000-1)*sum(df_all_iterations['Weight'])) )**(0.5)


TF2_variability = (sum(df_all_iterations['Weight*(TF2 - yBARw)^2'])*1000/(1000*(1000-1)*sum(df_all_iterations['Weight'])) )**(0.5)

print(df_all_iterations['TF1'][MSEs.index(min(MSEs))], df_all_iterations['TF2'][MSEs.index(min(MSEs))])
print(round(TF1_variability,3), round(TF2_variability,3))
print('RMSE =',min(MSEs)**0.5)

0.9578592953487262 -0.412005726392873
0.015 0.004
RMSE = 0.5947331618869144
